# Memória, Variáveis, Comunicação e Estado de um Programa

Há várias possíveis estratégias para usar a memória de um computador para se realizar a comunicação entre partes dele -- por exemplo, entre duas funções -- e para se manter o **estado atual** de execução de um programa. Dentre elas, podemos destacar o uso de variáveis locais, globais e parâmetros entre funções.

Vamos fazer aqui um exercício nesse contexto e você será conduzido a experimentar cada uma dessas estratégias, mesmo que em algumas ocasiões não seja a forma recomendada de se desenvolver um programa.


# Exercício do Empréstimo

Em um financiamento com juros compostos e número de parcelas fixas parte-se dos seguintes parâmetros:
* `S` - valor da primeira parcela
* `N` - número de parcelas
* `J` - percentual de juros mensal

A primeira parcela a ser paga do financiamento é sempre igual a `S`. A partir daí é feita uma atualização mensal da parcela, em que cada nova parcela é calculada a partir da parcela do mês anterior, conforme a fórmula:

> Parcela<sub>mês</sub> = Parcela<sub>mês-1</sub> * (1 + `J` / 100)

O financiamento encerra quando as `N` parcelas são pagas.

Exemplo:
* `S`: 200
* `N`: 5
* `J`: 1%

Parcelas do financiamento:
`200`; `202`; `204.02`; `206.06`; `208.12`

## Exercício Parte 1 - Escrevendo um Programa

Dado o problema descrito, escreva um programa que calcule as parcelas de um empréstimo para os seguintes valores:
* `S`: 200
* `N`: 5
* `J`: 1%

Esses valores podem ser atribuídos na função `main`, não é necessária leitura pelo teclado. Entretanto, o programa deve ser genérico, ou seja, funcionará para qualquer outro valor atribuído às 3 variáveis.

In [1]:
#include <stdio.h>

int main() {
    float S = 200;
    float J = 1;
    int N = 5;
    float parcelas[N];
    parcelas[0] = S;
    printf("O valor da parcela 1 eh %.02f\n", parcelas[0]);
    int x = 1;
    while (x < N) {
        parcelas[x] = parcelas[x - 1] * (1 + J / 100);
        printf("O valor da parcela %d eh %.02f\n", x + 1, parcelas[x]);
        x += 1;
    }
    return 1;
}

O valor da parcela 1 eh 200.00
O valor da parcela 2 eh 202.00
O valor da parcela 3 eh 204.02
O valor da parcela 4 eh 206.06
O valor da parcela 5 eh 208.12


[C kernel] Executable exited with code 1

## Exercício Parte 2 - Escrevendo um Módulo (Função)

Reescreva o código acima de forma que seu programa faça uso de uma função `proximaParcela` que seja responsável pelo cálculo de cada parcela X do empréstimo.

Utilize as boas práticas de modularização que você aprendeu, evitando dependências do módulo com o programa em que está inserida. Considere que esta função poderia ser eventualmente reusada em outro programa e deveria ser possível fazê-lo sem que ela dependa de nada do programa em que irá se inserir.

A função deve apenas calcular uma única parcela em cada chamada, portanto, ficará a cargo da função principal que chama `proximaParcela` realizar o restante do processo, para que sejam apresentadas todas as parcelas do financiamento.

In [2]:
#include <stdio.h>

void proximaParcela(int x, float parcelas[], float S, float J, int N) {
    parcelas[x] = parcelas[x - 1] * (1 + J / 100);
}
    
int main() {
    float S = 200;
    float J = 1;
    int N = 5;
    float parcelas[N];
    parcelas[0] = S;
    printf("O valor da parcela 1 eh %.02f\n", parcelas[0]);
    int x = 1;
    while (x < N) {
        proximaParcela(x, parcelas, S, J, N);
        printf("O valor da parcela %d eh %.02f\n", x + 1, parcelas[x]);
        x += 1;
    }
    return 1;
}

O valor da parcela 1 eh 200.00
O valor da parcela 2 eh 202.00
O valor da parcela 3 eh 204.02
O valor da parcela 4 eh 206.06
O valor da parcela 5 eh 208.12


[C kernel] Executable exited with code 1

## Exercício Parte 3 - Minimizando os Parâmetros

Na solução anterior, você deve ter usado vários parâmetros para a comunicação entre o `main` com a função `proximaParcela`, evitando usar variáveis globais. Modifique a forma como você representa o empréstimo, de forma que a função `proximaParcela` receba sempre um único parâmetro capaz de representar todos os valores que estejam relacionados a um empréstimo.

In [3]:
#include <stdio.h>

typedef struct {
    float S;
    float J;
    int N;
    float parcela;
} Emprestimo;

Emprestimo emprestimo(float S, float J, int N) {
    Emprestimo novo_emprestimo;
    novo_emprestimo.S = S;
    novo_emprestimo.J = J;
    novo_emprestimo.N = N;
    novo_emprestimo.parcela = S;
    return novo_emprestimo;
}

float proximaParcela(Emprestimo *novo_emprestimo) {
   novo_emprestimo->parcela = novo_emprestimo->parcela * (1 + novo_emprestimo->J / 100);
   return novo_emprestimo->parcela;
}

int main() {
    Emprestimo novo_emprestimo;
    float S = 200;
    float J = 1;
    int N = 5;
    novo_emprestimo = emprestimo(S, J, N);
    float parcela_atual = proximaParcela(&novo_emprestimo);
    printf("O valor da parcela 1 eh %.02f\n", S);
    int x = 1;
    while (x < N) { 
        printf("O valor da parcela %d eh %.02f\n", x + 1, parcela_atual);
        parcela_atual = proximaParcela(&novo_emprestimo);
        x += 1;
    }
    return 1;
}

O valor da parcela 1 eh 200.00
O valor da parcela 2 eh 202.00
O valor da parcela 3 eh 204.02
O valor da parcela 4 eh 206.06
O valor da parcela 5 eh 208.12


[C kernel] Executable exited with code 1

## Exercício Parte 4 - Mantendo o Estado

A função que você codificou na solução anterior provavelmente não é capaz de manter o controle do que chamaremos de **estado do empréstimo**, ou seja, dados de qual a parcela corrente ou da última parcela calculada.

Modifique a função (módulo) `proximaParcela` acima de modo que ela seja capaz de guardar o **estado do empréstimo** por sua conta, sem que o `main` precise ficar informando este estado por parâmetros (por exemplo). Para isso serão permitidas as seguintes modificações:

1. você pode desmembrar a função em mais de uma, por exemplo, uma você chama no começo do empréstimo e outra a cada parcela;

2. você pode usar técnicas de uso de variáveis não recomendadas que geram dependência das funções com o programa em que ela está inserida.

Você deve organizar o código de tal maneira que o `main` informe para as funções os dados do financiamento apenas uma única vez e depois possa solicitar o cálculo da parcela subsequente sem informar tudo novamente. O ideal é que o main não tenha acesso ao modo como as funções vão representar o **estado do empréstimo**, para que toda a lógica do empréstimo fique dentro das funções.

In [5]:
#include <stdio.h>

typedef struct {
    float S;
    float J;
    int N;
    float parcela;
} Emprestimo;

Emprestimo emprestimo(float S, float J, int N) {
    Emprestimo novo_emprestimo;
    novo_emprestimo.S = S;
    novo_emprestimo.J = J;
    novo_emprestimo.N = N;
    novo_emprestimo.parcela = S;
    return novo_emprestimo;
}

float comecar_emprestimo(Emprestimo *novo_emprestimo) {
    return novo_emprestimo->parcela;
}

float proximaParcela(Emprestimo *novo_emprestimo) {
    novo_emprestimo->parcela = novo_emprestimo->parcela * (1 + novo_emprestimo->J / 100);
    float parcela_anterior = novo_emprestimo->parcela;
    return parcela_anterior;
}

int main() {
    Emprestimo novo_emprestimo;
    float S = 200;
    float J = 1;
    int N = 5;
    novo_emprestimo = emprestimo(S, J, N);
    float primeiro_emprestimo = comecar_emprestimo(&novo_emprestimo);
    printf("O valor da parcela 1 eh %.02f\n", primeiro_emprestimo);
    int x = 2;
    while (x <= N) {
        float proxima_parcela = proximaParcela(&novo_emprestimo);
        printf("O valor da parcela %d eh %.02f\n", x, proxima_parcela);
        x += 1;
    }
    return 1;
}

O valor da parcela 1 eh 200.00
O valor da parcela 2 eh 202.00
O valor da parcela 3 eh 204.02
O valor da parcela 4 eh 206.06
O valor da parcela 5 eh 208.12


[C kernel] Executable exited with code 1

## Exercício Parte 5 - Múltiplos Empréstimos

Considere que há múltiplos empréstimos que podem ser controlados em paralelo. A sua função `main` deve ser capaz de apresentar no console as parcelas de mais de um empréstimo de modo paralelo, mantendo um controle para cada **estado de empréstimo** separadamente. Você deve decidir qual das soluções tomará como ponto de partida, se deve modificar a função `main`, as funções de cálculo do empréstimo ou ambas para atender esse requisito da melhor forma possível.

Adote uma solução compacta e generalizável, de tal modo que comporte até 100 empréstimo e, cada novo empréstimo, só exija a informação dos parâmetros de partida, sem expansão do código.

Por exemplo, suponha os seguintes dois empréstimos em paralelo:

### Empréstimo 1
* `S`: 200
* `N`: 5
* `J`: 1%

### Empréstimo 2
* `S`: 500
* `N`: 7
* `J`: 2%

A saída esperada é:
~~~
Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08
~~~

O exemplo mostra dois empréstimos, mas a estrutura deve ser genérica o suficiente para controlar N empréstimos em paralelo (até o limite de 100).

O seu programa deve apresentar o exemplo acima como teste.

In [6]:
#include <stdio.h>

typedef struct {
    float S;
    float J;
    int N;
    float parcela;
    int x; // parcela atual
} Emprestimo;

Emprestimo emprestimo(float S, float J, int N) { // essa funcao atribui os valores desejados para os registros
    // da struct
    Emprestimo novo_emprestimo;
    novo_emprestimo.S = S;
    novo_emprestimo.J = J;
    novo_emprestimo.N = N;
    novo_emprestimo.parcela = S; // parcela inicia com o valor de S
    novo_emprestimo.x = 1; // primeira parcela
    return novo_emprestimo;
}

float comecar_emprestimo(Emprestimo *novo_emprestimo) {
    return novo_emprestimo->parcela;
}

float proximaParcela(Emprestimo *novo_emprestimo) {
    float parcela_anterior = 0; // atribui 0 para a variavel parcela_anterior apenas para evitar lixo
    if (novo_emprestimo->x < novo_emprestimo->N) {
        novo_emprestimo->parcela = novo_emprestimo->parcela * (1 + novo_emprestimo->J / 100);
        parcela_anterior = novo_emprestimo->parcela;
    } else {
        novo_emprestimo->parcela = 0;
    }
    novo_emprestimo->x += 1;
    return parcela_anterior;
}
    
int main() {
    Emprestimo emprestimo_1, emprestimo_2;
    emprestimo_1 = emprestimo(200, 1, 5);
    emprestimo_2 = emprestimo(500, 2, 7);
    float primeiro_emprestimo_1 = comecar_emprestimo(&emprestimo_1);
    float primeiro_emprestimo_2 = comecar_emprestimo(&emprestimo_2);
    printf("Emprestimo 1: parcela 1 eh %.02f\n", primeiro_emprestimo_1);
    printf("Emprestimo 2: parcela 1 eh %.02f\n", primeiro_emprestimo_2);
    int x = 2; 
    while (x < emprestimo_1.N || x <= emprestimo_2.N) {
        float proxima_parcela_1 = proximaParcela(&emprestimo_1);
        if (proxima_parcela_1 != 0) {
             printf("Emprestimo 1: parcela %d eh %.02f\n", x, proxima_parcela_1);
        }
        float proxima_parcela_2 = proximaParcela(&emprestimo_2);
        if (proxima_parcela_2 != 0) {
            printf("Emprestimo 2: parcela %d eh %.02f\n", x, proxima_parcela_2);
        }
        x += 1;
    }
    return 1;
}

Emprestimo 1: parcela 1 eh 200.00
Emprestimo 2: parcela 1 eh 500.00
Emprestimo 1: parcela 2 eh 202.00
Emprestimo 2: parcela 2 eh 510.00
Emprestimo 1: parcela 3 eh 204.02
Emprestimo 2: parcela 3 eh 520.20
Emprestimo 1: parcela 4 eh 206.06
Emprestimo 2: parcela 4 eh 530.60
Emprestimo 1: parcela 5 eh 208.12
Emprestimo 2: parcela 5 eh 541.22
Emprestimo 2: parcela 6 eh 552.04
Emprestimo 2: parcela 7 eh 563.08


[C kernel] Executable exited with code 1